In [1]:
import math
from tqdm import tqdm
import time
import numpy as np
import torch
import gpytorch
from matplotlib import pyplot as plt
from ucimlrepo import fetch_ucirepo 
from math import floor
from torch.utils.data import TensorDataset, DataLoader
from gpytorch.models import ApproximateGP
from gpytorch.variational import CholeskyVariationalDistribution
from gpytorch.variational import VariationalStrategy

# Make plots inline
%matplotlib inline

In [2]:
def downloader(uci_id):
    # fetch dataset 
    uci_download = fetch_ucirepo(id=uci_id)
    
    # data (as pandas dataframes) 
    X_data = uci_download.data.features
    # Drop category and date variables
    if uci_id == 1:
        X_data = X_data.drop(["Sex"], axis=1)
    if uci_id == 275:
        X_data = X_data.drop(["dteday"], axis=1)
    if uci_id == 374:
        X_data = X_data.drop(["date"], axis=1)
    if uci_id == 183:
        X_data = X_data.drop(["communityname"], axis=1)
        object_columns = X_data.select_dtypes(include=['object']).columns
        X_data = X_data.drop(columns=object_columns)

    y_data = uci_download.data.targets
    # select target for datasets with 2 targets
    if uci_id == 189:
        y_data = y_data["motor_UPDRS"]
    if uci_id == 713:
        y_data = y_data["verification.time"]

    y = y_data.squeeze()

    X = torch.tensor(X_data.values, dtype=torch.float32)
    y = torch.tensor(y.values, dtype=torch.float32)

    train_n = int(floor(0.8 * len(X)))
    train_x = X[:train_n, :].contiguous()
    train_y = y[:train_n].contiguous()

    test_x = X[train_n:, :].contiguous()
    test_y = y[train_n:].contiguous()

    # Create TensorDataset and DataLoader for training and test sets
    train_dataset = TensorDataset(train_x, train_y)
    test_dataset = TensorDataset(test_x, test_y)

    train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False)

    return train_x, train_y, test_x, test_y, train_loader, test_loader

def whole_process(train_x, train_y, test_x, test_y, train_loader, test_loader):

    data_dim = train_x.size(-1)

    class LargeFeatureExtractor(torch.nn.Sequential):
        def __init__(self):
            super(LargeFeatureExtractor, self).__init__()
            self.add_module('linear1', torch.nn.Linear(data_dim, 1000))
            self.add_module('relu1', torch.nn.ReLU())
            self.add_module('linear2', torch.nn.Linear(1000, 500))
            self.add_module('relu2', torch.nn.ReLU())
            self.add_module('linear3', torch.nn.Linear(500, 50))
            self.add_module('relu3', torch.nn.ReLU())
            self.add_module('linear4', torch.nn.Linear(50, 2))

    feature_extractor = LargeFeatureExtractor()

    class GPModel(ApproximateGP):
        def __init__(self, inducing_points, feature_extractor):

            variational_distribution = CholeskyVariationalDistribution(inducing_points.size(0))

            variational_strategy = VariationalStrategy(self, inducing_points, variational_distribution, learn_inducing_locations=True)

            super(GPModel, self).__init__(variational_strategy)
            self.mean_module = gpytorch.means.ConstantMean()
            self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel(ard_num_dims=2))
            self.feature_extractor = feature_extractor
            self.scale_to_bounds = gpytorch.utils.grid.ScaleToBounds(-1., 1.)

        def forward(self, x):
            projected_x = self.feature_extractor(x)
            projected_x = self.scale_to_bounds(projected_x)
            mean_x = self.mean_module(projected_x)
            covar_x = self.covar_module(projected_x)
            return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

    inducing_points = train_x[:50, :]
    model = GPModel(inducing_points=inducing_points, feature_extractor=feature_extractor)
    likelihood = gpytorch.likelihoods.GaussianLikelihood()

    num_epochs = 100
    start_time = time.time()

    model.train()
    likelihood.train()

    optimizer = torch.optim.Adam([
        {'params': likelihood.parameters()},
        {'params': model.parameters()}
    ], lr=0.01)
    # Our loss object. We're using the VariationalELBO
    mll = gpytorch.mlls.VariationalELBO(likelihood, model, num_data=train_y.size(0))


    epochs_iter = tqdm(range(num_epochs), desc="Epoch")
    for i in epochs_iter:
        # Within each iteration, we will go over each minibatch of data
        minibatch_iter = tqdm(train_loader, desc="Minibatch", leave=False)
        for x_batch, y_batch in minibatch_iter:
            optimizer.zero_grad()
            output = model(x_batch)
            loss = -mll(output, y_batch)
            minibatch_iter.set_postfix(loss=loss.item())
            loss.backward()
            optimizer.step()

    end_time = time.time()
    wall_time = end_time - start_time

    model.eval()
    likelihood.eval()
    means = torch.tensor([0.])
    with torch.no_grad():
        for x_batch, y_batch in test_loader:
            preds = model(x_batch)
            means = torch.cat([means, preds.mean.cpu()])
    means = means[1:]
    test_rmse = torch.sqrt(torch.mean((means - test_y.cpu()) ** 2))

    means = torch.tensor([0.])
    with torch.no_grad():
        for x_batch, y_batch in train_loader:
            preds = model(x_batch)
            means = torch.cat([means, preds.mean.cpu()])
    means = means[1:]
    train_rmse = torch.sqrt(torch.mean((means - train_y.cpu()) ** 2))

    print('Test RMSE: {}'.format(test_rmse))
    print('Train RMSE: {}'.format(train_rmse))
    print('Wall time: {:.2f} seconds'.format(wall_time))

    return test_rmse, train_rmse, wall_time

experiment_datasets = [1, 275, 477, 189, 713, 186, 374, 183, 291, 294]

for experiment in experiment_datasets:
    print("Experiment " + str(experiment))

    trains = []
    tests = []
    times = []
    train_x, train_y, test_x, test_y, train_loader, test_loader = downloader(experiment)
    for run in range(3):
        # set seeds
        torch.manual_seed(run)
        print("run: " + str(run))

        test_rmse, train_rmse, wall_time = whole_process(train_x, train_y, test_x, test_y, train_loader, test_loader)
        trains.append(train_rmse)
        tests.append(test_rmse)
        times.append(wall_time)
    
    print("mean train performance= "+ str(np.mean(trains)))
    print("mean test performance= "+ str(np.mean(tests)))
    print("mean wall time= "+ str(np.mean(times)))

    print("std train= "+ str(np.std(trains, ddof=1)))
    print("std test= "+ str(np.std(tests, ddof=1)))

Experiment 1
run: 0


Epoch: 100%|██████████| 100/100 [00:35<00:00,  2.80it/s]


Test RMSE: 1.8057324886322021
Train RMSE: 4.113190650939941
Wall time: 35.72 seconds
run: 1


Epoch: 100%|██████████| 100/100 [00:35<00:00,  2.83it/s]


Test RMSE: 1.8790353536605835
Train RMSE: 4.176767826080322
Wall time: 35.31 seconds
run: 2


...

Test RMSE: 99.52696228027344
Train RMSE: 100.47309112548828
Wall time: 75.28 seconds
run: 1


Epoch: 100%|██████████| 100/100 [01:16<00:00,  1.31it/s]


Test RMSE: 77.42213439941406
Train RMSE: 75.91244506835938
Wall time: 76.13 seconds
run: 2


Epoch: 100%|██████████| 100/100 [01:15<00:00,  1.32it/s]


Test RMSE: 89.241455078125
Train RMSE: 89.45455932617188
Wall time: 75.80 seconds
mean train performance= 88.613365
mean test performance= 88.73019
mean wall time= 75.73735968271892
std train= 12.301912
std test= 11.061279
